# KBMOD Demonstration

In [1]:
# First, confirm that we have GPU access and that it has the expected RAM
!nvidia-smi

Thu Jan 23 09:18:01 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  | 00000000:01:00.0 Off |                    0 |
| N/A   34C    P0              55W / 400W |      0MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
# Load general libraries needed
from astropy.time import Time
from astropy.wcs import WCS
from astropy.coordinates import EarthLocation

import os
homedir = '/sdf/home/c/colinc'

In [3]:
# Install kbmod capable of running in this Jupyter session
cwd = os.getcwd()
os.chdir(homedir)
kbmod_usdf_dir = 'kbmod-usdf'
os.makedirs(kbmod_usdf_dir,exist_ok=True)
os.chdir(kbmod_usdf_dir)
#!get_kbmod_again.sh
os.chdir('kbmod')
!pip install .
os.chdir(cwd)

Processing /sdf/home/c/colinc/kbmod-usdf/kbmod
  Installing build dependencies ... one
  Getting requirements to build wheel ... one
  Preparing metadata (pyproject.toml) ... one
  Created wheel for kbmod: filename=kbmod-1.1.1.dev1328+gb7e3ceff-cp311-cp311-linux_x86_64.whl size=832840 sha256=e3d886e3803c39f13962df05b4366d40bb3d22a6449a35dcbdf8b1ff491e9a51
  Stored in directory: /lscratch/colinc/pip-ephem-wheel-cache-6p9qnkl3/wheels/fa/50/f3/f10f5e1f90667941c1d13952117872c037d38a5da63eadfbd8
Successfully built kbmod
  Attempting uninstall: kbmod
    Found existing installation: kbmod 1.1.1.dev1301+g74366154
    Uninstalling kbmod-1.1.1.dev1301+g74366154:
      Successfully uninstalled kbmod-1.1.1.dev1301+g74366154


In [19]:
# import the neeeded kbmod components
import kbmod
import kbmod.reprojection as reprojection
from kbmod.results import Results
from kbmod.analysis.plotting import *
from kbmod.analysis.visualizer import Visualizer
from kbmod.configuration import SearchConfiguration

In [5]:
# Paths and filename strings we will need

# This is an ImageCollection, output by kbmod.ImageCollection
colfile = '/sdf/home/c/colinc/rubin-user/20X20_patch250060/tno_2013_TA228_ComCam_w50_20x20_patch250060_betterIDs.collection'

# This is the Work Unit that contains both metadata and image data
# Presently, this must be generated from an interactive session on a USDF compute node capable of accessing both the USDF Butler and kbmod
wu_file = 'tno_2013_TA228_ComCam_w50_20x20_patch110571_wu.fits'

# The working directory where images and other input and output files will be located
workdir = os.path.dirname(colfile) # '/sdf/home/c/colinc/rubin-user/20X20_patch250060'
os.makedirs(workdir, exist_ok=True)

configfile = f'{workdir}/search_config_deep_20250122_39au_a.yaml'

#  directory_containing_shards, wu_filename = os.path.split(f'{workdir}/{wu_file}')

repro_shard_dir = os.path.join(workdir, 'repro') # directory_containing_shards+'/repro'
os.makedirs(repro_shard_dir, exist_ok=True)

In [6]:
# this is the "guess distance" in astronomical units for our reflex-correction and reprojection routines
guess_dist = 39 # au

In [8]:
# Load the ImageCollection
ic = kbmod.ImageCollection.read(colfile)
print(f'Read {len(ic)} rows from {colfile}.')

Read 249 rows from /sdf/home/c/colinc/rubin-user/20X20_patch250060/tno_2013_TA228_ComCam_w50_20x20_patch250060_betterIDs.collection.


In [9]:
# Example row from the ImageCollection
ic[0]

dataId,visit,detector,band,filter,mjd_start,mjd_mid,object,pointing_ra,pointing_dec,airmass,wcs_err,ra,dec,ra_bl,dec_bl,ra_tl,dec_tl,ra_tr,dec_tr,ra_br,dec_br,psfSigma,psfArea,nPsfStar,zeroPoint,skyBg,skyNoise,meanVar,OBSID,wcs,exposureTime,collection,datasetType,obs_lon,obs_lat,obs_elev,astromOffsetMean,astromOffsetStd,DIMM2SEE,global_wcs,global_wcs_pixel_shape_0,global_wcs_pixel_shape_1
str36,int64,int64,str1,str4,float64,float64,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,str20,str1248,float64,str56,str28,float64,float64,float64,float64,float64,float64,str2880,int64,int64
615b2f69-ba41-4d4b-b29b-a807e7a3ac82,2024112600134,8,i,i_06,60641.08122699644,60641.08140639459,Rubin_SV_38_7,38.0653955028628,6.52826805904292,1.28850130948918,-3.090860900556436e-12,38.18728959931873,6.2575399328953285,38.13367905883207,6.407125257662862,38.03592432637463,6.206883852393906,38.24092893534934,6.107783069946998,38.33871170185693,6.30809586582424,2.1870157927318044,67.46518367993608,102,31.76327910494223,2581.0746131539345,52.993192583897475,2795.876509900088,CC_O_20241126_000134,"{""WCSAXES"":2,""CRPIX1"":2104.234496,""CRPIX2"":2090.780895,""PC1_1"":5.0049364571068e-05,""PC1_2"":-2.4296678395023e-05,""PC2_1"":-2.4329910755198e-05,""PC2_2"":-5.0071238405668e-05,""CDELT1"":1.0,""CDELT2"":1.0,""CUNIT1"":""deg"",""CUNIT2"":""deg"",""CTYPE1"":""RA---TAN-SIP"",""CTYPE2"":""DEC--TAN-SIP"",""CRVAL1"":38.188480325674,""CRVAL2"":6.2514088148071,""LONPOLE"":180.0,""LATPOLE"":6.2514088148071,""MJDREF"":0.0,""RADESYS"":""ICRS"",""A_ORDER"":2,""A_0_2"":5.71380993437748e-08,""A_1_1"":8.4138548688458e-08,""A_2_0"":1.34602072087139e-07,""B_ORDER"":2,""B_0_2"":1.29922197020279e-07,""B_1_1"":9.30259268861285e-08,""B_2_0"":5.08093732261702e-08,""AP_ORDER"":3,""AP_0_0"":2.38650612711241e-07,""AP_0_1"":-1.9773730332761e-11,""AP_0_2"":-5.7138214915058e-08,""AP_0_3"":1.96568954030525e-14,""AP_1_0"":-2.9495295095216e-11,""AP_1_1"":-8.4138741264852e-08,""AP_1_2"":4.40296314635341e-14,""AP_2_0"":-1.3460222841301e-07,""AP_2_1"":4.76169097340427e-14,""AP_3_0"":4.05162046764094e-14,""BP_ORDER"":3,""BP_0_0"":2.2872495540209e-07,""BP_0_1"":-2.0712098702801e-11,""BP_0_2"":-1.2992234828618e-07,""BP_0_3"":3.90781540503791e-14,""BP_1_0"":-2.15361705159e-11,""BP_1_1"":-9.3026118127959e-08,""BP_1_2"":4.98983852539292e-14,""BP_2_0"":-5.080948323256e-08,""BP_2_1"":4.29343119560157e-14,""BP_3_0"":1.84076385356384e-14,""NAXIS1"":4072,""NAXIS2"":4000}",30.0,LSSTComCam/runs/DRP/20241101_20241211/w_2024_50/DM-48128,goodSeeingDiff_differenceExp,-70.7494170285165,-30.2446389756252,2662.99616375123,nan,nan,0.0,"WCSAXES = 2 / Number of coordinate axes CRPIX1 = 3000.0 / Pixel coordinate of reference point CRPIX2 = 3000.0 / Pixel coordinate of reference point CDELT1 = -5.5555555555556E-05 / [deg] Coordinate increment at reference point CDELT2 = 5.5555555555556E-05 / [deg] Coordinate increment at reference point CUNIT1 = 'deg' / Units of coordinate increment and value CUNIT2 = 'deg' / Units of coordinate increment and value CTYPE1 = 'RA---TAN' / Right ascension, gnomonic projection CTYPE2 = 'DEC--TAN' / Declination, gnomonic projection CRVAL1 = 38.666666666667 / [deg] Coordinate value at reference point CRVAL2 = 6.8333333333333 / [deg] Coordinate value at reference point LONPOLE = 180.0 / [deg] Native longitude of celestial pole LATPOLE = 6.8333333333333 / [deg] Native latitude of celestial pole MJDREF = 0.0 / [d] MJD of fiducial time RADESYS = 'ICRS' / Equatorial coordinate system END",6000,6000


In [12]:
def update_ic_meta(ic):
    ic.meta['n_stds'] = len(ic)
    ic.data['std_idx'] = range(len(ic))
    return ic

In [13]:
ic = update_ic_meta(ic)
ic.write(colfile, overwrite=True)

In [14]:
# If the Jupyter server was capable of generating the WorkUnit, this would be the code to run.
# search_config_filepath = configfile
# wu = ic.toWorkUnit(search_config=SearchConfiguration.from_file(search_config_filepath), butler=butler)

In [16]:
%%time
wu = kbmod.work_unit.WorkUnit.from_sharded_fits(wu_file, workdir, lazy=False) # directory_containing_shards
print(f'Loaded {len(wu)} images from {wu_filename}.')

CPU times: user 11.3 s, sys: 25.9 s, total: 37.2 s
Wall time: 37.7 s


## Create reflex-corrected WCS for each image

In [25]:
%%time
#! This method to get image dimensions won't hold if the images are different sizes.
image_height, image_width = wu.get_wcs(0).array_shape

# Find the estimated barycentric distance (EBD) world coordinate system (WCS) for each image
ebd_per_image_wcs, geocentric_dists = kbmod.reprojection_utils.transform_wcses_to_ebd(
    [wu.get_wcs(i) for i in range(len(wu))],
    width=image_width,
    height=image_height,
    heliocentric_distance=guess_dist, # heliocentric guess distance in AU
    obstimes = Time(wu.get_all_obstimes(), format="mjd"),
    point_on_earth=EarthLocation.of_site("Rubin Observatory"),
    npoints=10,
    # seed=None,
)

# Updating WorkUnit metadata
wu.org_img_meta["ebd_wcs"] = ebd_per_image_wcs
wu.heliocentric_distance = guess_dist
wu.org_img_meta["geocentric_distance"] = geocentric_dists

CPU times: user 25.8 s, sys: 56.6 ms, total: 25.9 s
Wall time: 26.2 s


## Reprojecting images to common WCS
#### Here, we create mosaics from the input images and handle arbitrary image rotation.

In [26]:
%%time
common_wcs = WCS(ic.data['global_wcs'][0])
common_wcs.pixel_shape = (ic.data['global_wcs_pixel_shape_0'][0], ic.data['global_wcs_pixel_shape_1'][0])

reprojected_wu = reprojection.reproject_work_unit(
    wu,
    common_wcs,
    parallelize=True,
    frame="ebd",
    # max_parallel_processes=8,
)

# assign the prescribed guess distance to the WorkUnit metadata.
reprojected_wu.heliocentric_distance = guess_dist

Reprojecting: 100%|██████████| 57/57 [41:12]


CPU times: user 18.6 s, sys: 1min 17s, total: 1min 36s
Wall time: 41min 22s


In [23]:
repro_workunit_filename = wu_file + '.repro'

In [28]:
# Save the reprojected WorkUnit
reprojected_wu.to_sharded_fits(repro_workunit_filename, repro_shard_dir, overwrite=True)

In [15]:
# Load the previously saved reprojected WorkUnit
reprojected_wu = kbmod.work_unit.WorkUnit.from_sharded_fits(repro_workunit_filename, repro_shard_dir, lazy=False)
reprojected_wu.heliocentric_distance = guess_dist # TODO: remove

In [20]:
# Modify and update the kbmod configuration based on the .yaml file.
config = kbmod.configuration.SearchConfiguration.from_file(configfile)
resultsdir = f'{workdir}/results'
os.makedirs(resultsdir, exist_ok=True)
full_results = os.path.join(resultsdir, 'full_results.ecsv')
reprojected_wu.config = config

NameError: name 'resampled_wu' is not defined

## Run the KBMOD search

In [17]:
%%time
res = kbmod.run_search.SearchRunner().run_search_from_work_unit(reprojected_wu)
res.write_table(full_results)
print(f'Found {len(res)} results spanning {len(reprojected_wu} unique timestamps/images.')

CPU times: user 30min 55s, sys: 4min 5s, total: 35min 1s
Wall time: 31min 31s


## Visualize Results
#### Here we visualize results in the form of co-added image cutouts and display the results table.

In [2]:
# choose which flavor of result image to visualize
embargo_ok = False
if embargo_ok == True:
    res.table["stamp"] = res.table["coadd_weighted"]
    
    viz = Visualizer(reprojected_wu.im_stack, res)
    viz.generate_all_stamps()
    viz.count_num_days()
    
    for res_idx in range(len(res)):
        viz.plot_daily_coadds(res_idx)
else:
    print(f'Image data will display here during the live demo.')

Image data will display here during the live demo.


In [18]:
# Display the results table
res.table

NameError: name 'res' is not defined

In [ ]:
# TODO: add matching to known objects